In [175]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle
import torch
import pandas as pd
import math
from sklearn.model_selection import train_test_split

from transformers import Wav2Vec2Processor, Wav2Vec2FeatureExtractor, HubertModel, HubertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import soundfile as sf


from IPython.display import Audio, display, clear_output

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [176]:
import sys
sys.path.append("../scripts/")
import data_loader as dl

In [153]:
p_path =  "C:/Users/yagne/Downloads/feature_extracts_11/"
data = pd.read_csv("../outputs/summary_data.csv")
data['line'] = data['line'].astype(str)
data.sample(n=4)


,file,line,start,end,speaker,speaker_role,word_count,duration,text,start_idx,...,justice_year_onset_time_diff_mean_mean,justice_year_onset_time_diff_mean_std,justice_year_onset_time_diff_stddev_mean,justice_year_onset_time_diff_stddev_std,justice_year_onset_time_diff_entropy_mean,justice_year_onset_time_diff_entropy_std,justice_year_word_rate_mean,justice_year_word_rate_std,justice_year_onset_rate_mean,justice_year_onset_rate_std
4304,14-1373,44,788.900,828.110,Sonia_Sotomayor,scotus_justice,99,39.210,This is a non sequitur. When you talk about an...,12622400,...,0.206733,0.035625,0.217872,0.048200,-74.549480,37.873219,2.348692,0.373659,6.261980,0.891660
7996,16-1144,78,996.800,1012.240,Stephen_G_Breyer,scotus_justice,43,15.440,So your argument is nexus to a pending proceed...,15948800,...,0.202739,0.039889,0.194595,0.052545,-95.838989,64.630880,2.862498,0.496945,6.545924,1.008240
7635,15-8544,165,3740.510,3748.970,John_G_Roberts_Jr,scotus_justice,40,8.460,"Mr. Mortara, this Court appointed you to brief...",59848160,...,0.161744,0.022075,0.137940,0.028584,-158.519116,80.777301,3.243645,0.503014,7.711613,0.775075
1486,12-7822,100,1994.447,2027.168,Elena_Kagan,scotus_justice,94,32.721,"You see, I thought that Randolph rejected that...",31911152,...,0.178505,0.027986,0.151338,0.030079,-136.943023,56.329519,2.982173,0.376701,7.209296,0.770327


In [58]:
mp3_clips = pickle.load(open(f"C:/Users/yagne/Downloads/mp3_tensors.pkl", "rb"))

In [64]:
mp3_clips = np.array([(m['file'], m['line']) for m in mp3_clips])
# mp3_clips = np.array(mp3_clips)

In [126]:
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
sampling_rate = dataset.features["audio"].sampling_rate

Reusing dataset librispeech_asr (C:\Users\yagne\.cache\huggingface\datasets\hf-internal-testing___librispeech_asr\clean\2.1.0\f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc)


In [127]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('superb/hubert-base-superb-ks')
model = HubertForSequenceClassification.from_pretrained('superb/hubert-base-superb-ks')

In [128]:
# audio file is decoded on the fly
inputs = feature_extractor(dataset[0]["audio"]["array"], return_tensors="pt")
logits = model(**inputs).logits
predicted_class_ids = torch.argmax(logits, dim=-1).detach().numpy()[0]
predicted_label = model.config.id2label[predicted_class_ids]
# compute loss - target_label is e.g. "down"
target_label = model.config.id2label[3]
inputs["labels"] = torch.tensor([model.config.label2id[target_label]])
loss = model(**inputs).loss

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
C:\Users\yagne\.conda\envs\grandstanding\lib\site-packages\transformers\models\hubert\modeling_hubert.py:803: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


In [134]:
check = model(**inputs)

C:\Users\yagne\.conda\envs\grandstanding\lib\site-packages\transformers\models\hubert\modeling_hubert.py:803: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


In [144]:
check.hidden_states

(tensor([[[-0.2746, -0.1840,  0.0569,  ..., -0.1526,  0.2688, -0.3192],
          [-0.3161, -0.2185, -0.0018,  ...,  0.2225,  0.2545, -0.3387],
          [-0.1887, -0.2121, -0.0785,  ..., -0.0594,  0.1773, -0.1694],
          ...,
          [-0.1011, -0.0561, -0.0506,  ..., -0.1106,  0.3118, -0.1729],
          [-0.0458, -0.0886, -0.0198,  ...,  0.1656,  0.2204, -0.3392],
          [-0.1749, -0.0031,  0.1134,  ...,  0.0410,  0.1449, -0.3321]]],
        grad_fn=<NativeLayerNormBackward0>),
 tensor([[[-0.1023,  0.0447,  0.0982,  ..., -0.0698,  0.0521,  0.0044],
          [-0.1448, -0.0155,  0.0404,  ..., -0.0147,  0.0598, -0.1860],
          [ 0.0181,  0.0223, -0.0565,  ..., -0.1626, -0.0226, -0.0382],
          ...,
          [ 0.0196, -0.0267, -0.1686,  ..., -0.0993,  0.1906, -0.2106],
          [-0.0582, -0.0835, -0.3201,  ..., -0.0952,  0.1258, -0.2843],
          [-0.1376,  0.1162, -0.1144,  ..., -0.0488,  0.0041, -0.2616]]],
        grad_fn=<NativeLayerNormBackward0>),
 tensor([[[ 

In [20]:
print(dataset[0]["audio"]["array"])
print(inputs)
print(logits)
print(predicted_class_ids)
print(predicted_label)
print(target_label)
print(inputs)
print(loss)

[0.00238037 0.0020752  0.00198364 ... 0.00042725 0.00057983 0.0010376 ]
{'input_values': tensor([[0.0024, 0.0021, 0.0020,  ..., 0.0004, 0.0006, 0.0010]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), 'labels': tensor([3])}
tensor([[ -0.4311,   1.1562,  -7.0366,  -1.3205,  -0.6599,  -1.4566,  -4.0140,
          -4.6705,  -2.0571,   2.8500,   8.0967, -15.1642]],
       grad_fn=<AddmmBackward0>)
10
_unknown_
down
{'input_values': tensor([[0.0024, 0.0021, 0.0020,  ..., 0.0004, 0.0006, 0.0010]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), 'labels': tensor([3])}
tensor(9.4240, grad_fn=<NllLossBackward0>)


In [12]:
model.config.id2label

{0: 'yes',
 1: 'no',
 2: 'up',
 3: 'down',
 4: 'left',
 5: 'right',
 6: 'on',
 7: 'off',
 8: 'stop',
 9: 'go',
 10: '_unknown_',
 11: '_silence_'}

In [17]:
Audio(data=dataset[0]["audio"]["array"],rate=sampling_rate)

In [26]:
inputs['attention_mask']

tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)

In [40]:
mp3_clips[0]
inthis = mp3_clips[0]
attention_mask = torch.ones(inthis['clip'].size(), dtype=torch.int32)
b=data.loc[(data['file'] == inthis['file']) & (data['line'] == inthis['line']),'gs_score'].values[0]
torch.tensor(b).unsqueeze(0)

tensor([-0.7607], dtype=torch.float64)

In [185]:
class HUBERTDataset(torch.utils.data.Dataset):
    def __init__(self, mp3_clips, summary_data, pt_path):
        self.mp3_clips = mp3_clips
        self.summary_data = summary_data
        self.pt_path = pt_path

    def __getitem__(self, idx):
        file, line = self.mp3_clips[idx]
        input_values = torch.load(f"{self.pt_path}{file}-{line}.pt")
        input_values.requires_grad = False
        input_values = input_values.squeeze(0)
        labels = self.summary_data.loc[
            (self.summary_data['file'] == file) & (self.summary_data['line'] == line),'gs_score'
        ].values[0]
        labels = torch.tensor(labels).unsqueeze(0)
        return {'input_values': input_values,
                'labels': labels
               }

    def __len__(self):
        return len(self.mp3_clips)

In [186]:
print(next(iter(HUBERTDataset(train_clips, data, "C:/Users/yagne/Downloads/feature_extracts_11/"))))
print(next(iter(HUBERTDataset(val_clips, data, "C:/Users/yagne/Downloads/feature_extracts_11/"))))
a = next(iter(HUBERTDataset(test_clips, data, "C:/Users/yagne/Downloads/feature_extracts_11/")))
print(a)

{'input_values': tensor([[ 0.1276,  0.3550,  0.1032,  ..., -0.2386,  0.1234, -1.5782],
        [ 0.0027,  0.3684,  0.0602,  ..., -0.1701,  0.0677, -1.4073],
        [-0.1403,  0.2992,  0.1498,  ...,  0.2281,  0.2809, -0.4651],
        ...,
        [-0.1259, -0.1543, -0.4424,  ...,  0.0028, -0.2310,  0.3080],
        [-0.0830, -0.0977, -0.3098,  ..., -0.0694, -0.0440,  0.0494],
        [-0.0609, -0.0810, -0.1101,  ...,  0.0220,  0.1004, -0.0365]]), 'labels': tensor([0.2023], dtype=torch.float64)}
{'input_values': tensor([[-7.0643e-02,  2.1688e-01,  3.7034e-01,  ...,  6.3092e-04,
          2.7395e-01,  6.5991e-01],
        [-4.3117e-02,  1.8276e-01,  3.2171e-01,  ..., -5.9538e-02,
          2.3518e-01,  7.3286e-01],
        [ 1.9535e-01, -2.6284e-01,  2.8479e-01,  ..., -5.0804e-02,
          4.8672e-02,  6.0160e-01],
        ...,
        [ 1.7206e-01,  1.8350e-01,  6.8635e-02,  ..., -4.3249e-01,
         -3.2615e-02,  1.7416e-01],
        [-2.2166e-01,  2.3964e-01,  3.8822e-01,  ..., -1.

In [167]:
a['input_values']

tensor([[[-3.4103e-02,  1.0014e-01,  2.4641e-01,  ..., -8.9486e-02,
           1.6354e-01,  5.9500e-01],
         [-2.9758e-02,  2.9770e-01,  2.7752e-01,  ..., -1.2697e-01,
          -7.0912e-03,  8.2249e-01],
         [ 2.0726e-01, -1.1702e-01,  2.0951e-02,  ..., -1.0313e-01,
          -1.6401e-01,  1.1113e+00],
         ...,
         [-7.9326e-02, -1.2018e-02,  3.7434e-02,  ..., -2.6063e-01,
          -3.9066e-04, -2.4367e-01],
         [-2.9478e-01,  1.4937e-01,  1.8876e-01,  ..., -6.4171e-02,
           1.4794e-01, -1.2063e+00],
         [ 1.9130e-01, -2.8415e-01, -8.8065e-02,  ..., -3.7400e-01,
           4.1668e-02, -1.6149e+00]]])

In [106]:
file, line = mp3_clips[0]
file, line

('13-502', '2')

In [73]:
train_clips, test_clips = train_test_split(mp3_clips, test_size=0.1)
train_clips, val_clips = train_test_split(train_clips, test_size=0.1)

In [74]:
print(train_clips.shape, val_clips.shape, test_clips.shape)

(2681, 2) (298, 2) (331, 2)


In [18]:
Audio(test_clips[1]['clip'], rate=16000)

In [45]:
a = model(input_values)

In [47]:
a[0]

tensor([[[ 1.0639e-01,  4.2007e-01,  4.7534e-01,  ...,  1.9919e-01,
          -2.1368e-01, -6.6571e-02],
         [ 7.9914e-02,  3.8841e-01,  5.8362e-01,  ...,  2.0241e-01,
          -1.8895e-01,  1.2788e-01],
         [ 8.1476e-02,  7.1193e-01, -3.9917e-02,  ..., -2.0461e-01,
          -2.5947e-01, -2.1147e-01],
         ...,
         [ 1.1822e-01,  3.5618e-01,  4.9075e-01,  ...,  1.6532e-01,
          -3.3431e-01,  2.4377e-01],
         [-4.0234e-04,  4.1054e-01,  4.0781e-01,  ...,  1.7384e-01,
          -3.5164e-01,  1.6486e-01],
         [-4.2161e-02,  4.1631e-01,  2.8500e-01,  ...,  2.0109e-01,
          -3.4025e-01,  1.4237e-03]]], grad_fn=<NativeLayerNormBackward0>)

In [51]:
b = processor(ds["speech"][0], return_tensors="pt")

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [65]:
a[0].size()

torch.Size([1, 292, 1024])

In [117]:
a['input_values'].size()

torch.Size([1, 999, 768])

In [118]:
from transformers import Wav2Vec2Processor, HubertModel
from datasets import load_dataset
import soundfile as sf
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertModel.from_pretrained("facebook/hubert-large-ls960-ft")
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)
input_values = processor(ds["speech"][0], return_tensors="pt").input_values  # Batch size 1
hidden_states = model(input_values).last_hidden_state

Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Reusing dataset librispeech_asr (C:\Users\yagne\.cache\huggingface\datasets\hf-internal-testing___librispeech_asr\clean\2.1.0\f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc)
Loading cached processed dataset at C:\Users\yagne\.cache\huggingface\datasets\hf-internal-testing___librispeech_asr\clean\2.1.0\f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc\cache-

In [121]:
hidden_states.size()

torch.Size([1, 292, 1024])

In [146]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

In [190]:
class HUBERTDataModule(pl.LightningDataModule):
    def __init__(self, mp3_clips, metadata, pt_path, batch_size=4, num_workers=4, seed=42):
        super().__init__()
        self.mp3_clips = mp3_clips
        self.metadata = metadata
        self.pt_path = pt_path
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.seed = seed
        
    def prepare_data(self):
        rng = np.random.default_rng(self.seed)
        indices = rng.permutation(self.mp3_clips.shape[0])
        train_size = math.floor(len(indices) * 0.80)
        val_size = math.floor(len(indices) * 0.10)   
        self.train_idx = indices[:train_size]
        self.val_idx = indices[train_size : train_size + val_size]
        self.test_idx = indices[train_size + val_size :]
        
    def setup(self):
        self.train_clips = self.mp3_clips[self.train_idx]
        self.val_clips = self.mp3_clips[self.val_idx]
        self.test_clips = self.mp3_clips[self.test_idx]
        
    def train_dataloader(self):
        self.train_data = dl.HUBERTDataset(self.train_clips, self.metadata, self.pt_path)
        train_loader = DataLoader(self.train_data, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)
        return train_loader
    def val_dataloader(self):
        self.val_data = dl.HUBERTDataset(self.val_clips, self.metadata, self.pt_path)
        val_loader = DataLoader(self.val_data, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)
        return val_loader
    def test_dataloader(self):
        self.test_data = dl.HUBERTDataset(self.test_clips, self.metadata, self.pt_path)
        test_loader = DataLoader(self.test_data, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)
        return test_loader
        

In [191]:
dm = HUBERTDataModule(mp3_clips, data, p_path)
dm.prepare_data()
dm.setup()

b1 = next(iter(dm.train_dataloader()))

In [194]:
b1.keys()

dict_keys(['input_values', 'labels'])

In [ ]:
class HubertForSequenceClassification(HubertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        self.hubert = HubertModel(config)
        num_layers = config.num_hidden_layers + 1  # transformer layers + input embeddings
        if config.use_weighted_layer_sum:
            self.layer_weights = nn.Parameter(torch.ones(num_layers) / num_layers)
        self.projector = nn.Linear(config.hidden_size, config.classifier_proj_size)
        self.classifier = nn.Linear(config.classifier_proj_size, config.num_labels)

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        input_values,
        attention_mask=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        labels=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss. Indices should be in :obj:`[0, ...,
            config.num_labels - 1]`. If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """

        return_dict = return_dict
        output_hidden_states = output_hidden_states

        # Already ran this
        outputs = self.hubert(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        hidden_states = outputs[0]

        hidden_states = self.projector(hidden_states)
        pooled_output = hidden_states.mean(dim=1)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))


        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [207]:
class HUBERTRegressor(pl.LightningModule):
    def __init__(self, criterion, learning_rate=0.0001, hidden_size=768, projector_size=256):
        super().__init__()
        self.projector = nn.Linear(hidden_size, projector_size)
        self.regressor = nn.Linear(projector_size, 1)
        self.criterion = criterion
        self.learning_rate = learning_rate
        
    def forward(self, x):
        print("step1")
        projected = self.projector(x)
        print("step2")
        pooled_output = projected.mean(dim=1)
        print("step3")
        pred = self.regressor(pooled_output)
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)
    
    def training_step(self, batch, batch_idx):
        x, y = batch['input_values'], batch['labels']
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch['input_values'], batch['labels']
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch['input_values'], batch['labels']
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss

In [211]:
checkpoint_callback = ModelCheckpoint(
    dirpath = "checkpoints",
    filename = "best-checkpoint", 
    save_top_k=1, 
    verbose =True, 
    monitor = "val_loss",
    mode="min"
)
logger = TensorBoardLogger("hubert")
early_stopping_callback = EarlyStopping(monitor="val_loss", patience = 0)

trainer = pl.Trainer(
    max_epochs=5,
    logger=logger,
    gpus=0,
    checkpoint_callback=checkpoint_callback,
    callbacks = [early_stopping_callback],
#     overfit_batches=1
)

C:\Users\yagne\.conda\envs\grandstanding\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x0000023261CBBD60>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x0000023261CBBD60>)`.
  rank_zero_deprecation(
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
C:\Users\yagne\.conda\envs\grandstanding\lib\site-packages\pytorch_lightning\trainer\trainer.py:1579: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(


In [212]:
model = HUBERTRegressor(criterion=nn.MSELoss())

In [ ]:
trainer.fit(model, dm)
trainer.test(model, datamodule=dm)


  | Name      | Type    | Params
--------------------------------------
0 | projector | Linear  | 196 K 
1 | regressor | Linear  | 257   
2 | criterion | MSELoss | 0     
--------------------------------------
197 K     Trainable params
0         Non-trainable params
197 K     Total params
0.788     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

In [205]:
model.to("cpu")

HUBERTRegressor(
  (projector): Linear(in_features=768, out_features=256, bias=True)
  (regressor): Linear(in_features=256, out_features=1, bias=True)
  (criterion): MSELoss()
)

In [206]:
model(b1['input_values'])